In [ ]:
import cv2
import face_recognition
import datetime

def check_faces(known_face_encodings, known_face_names):

    # Initialize the webcam
    video_capture = cv2.VideoCapture(1)

    while True:
        # Capture a single frame of video
        ret, frame = video_capture.read()

        # Find all the faces in the current frame of video
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)

        # Loop through each face in this frame of video
        for face_encoding, face_location in zip(face_encodings, face_locations):
            # See if the face is a match for the known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Find the best match
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = face_distances.argmin()
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Draw a rectangle around the face
            top, right, bottom, left = face_location
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with the name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            # Check if it's before noon
            if datetime.datetime.now().hour < 12:
                check_type = "Check-in"
            else:
                check_type = "Check-out"

            # Log the check-in/check-out time and name
            with open("log.txt", "a") as f:
                f.write(f"{check_type}: {name} - {datetime.datetime.now()}\n")

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Exit the program if the user presses 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close the window
    video_capture.release()
    cv2.destroyAllWindows()


In [ ]:
import face_recognition
person1_image = face_recognition.load_image_file("com.jpg")
person2_image = face_recognition.load_image_file("2.jpg")
person3_image = face_recognition.load_image_file("chuan.jpg")
person4_image = face_recognition.load_image_file("chitsata.jpg")
person5_image = face_recognition.load_image_file("spy.jpg")



person1_encoding = face_recognition.face_encodings(person1_image)[0]
person2_encoding = face_recognition.face_encodings(person2_image)[0]
person3_encoding = face_recognition.face_encodings(person3_image)[0]
person4_encoding = face_recognition.face_encodings(person4_image)[0]
person5_encoding = face_recognition.face_encodings(person5_image)[0]
known_face_encodings = [person1_encoding,person2_encoding,person3_encoding,person4_encoding,person5_encoding]
known_face_names = [ "Com","Barack Obama","Chuan","Chitsata","Spy"]


with open("encodings.txt", "w") as f:
    for encoding, name in zip(known_face_encodings, known_face_names):
        f.write("{}\n{}\n".format(name, encoding))

In [10]:
import face_recognition
import os
# assign directory
directory = './flask_workshop/static/'
known_face_encodings = []
known_face_names =[]
# iterate over files in
# that directory
for filename in os.listdir(directory):
    known_face_names.append(filename.replace(".jpg",""))
    f = os.path.join(directory, filename)
    person_image = face_recognition.load_image_file(f)
    person_encoding = face_recognition.face_encodings(person_image)[0]
    known_face_encodings.append(person_encoding)

['Spy', 'Chuan', 'Com', 'Chitsata', 'Barak']


In [12]:
import cv2
import face_recognition
from datetime import datetime
import numpy as np

# known_face_names = []
# known_face_encodings = []

# Initialize the webcam
try:
    video_capture = cv2.VideoCapture(0)
except:
    video_capture = cv2.VideoCapture(1)
# Initialize variables for check-in and check-out times
check_in_times = {}
check_out_times = {}

while True:
    # Capture a single frame of video
    ret, frame = video_capture.read()

    # Find all the faces in the current frame of video
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    # Loop through each face in this frame of video
    for face_encoding, face_location in zip(face_encodings, face_locations):
        # See if the face is a match for the known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        # Find the best match
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = face_distances.argmin()
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

            # Get the current date and time
            now = datetime.now()

            # Check if it's morning or afternoon
            if now.hour < 12:
                time_of_day = "morning"
            else:
                time_of_day = "afternoon"

            # Check if it's the first time the person is checking in
            if name not in check_in_times and time_of_day =="morning":
                check_in_times[name] = now
                print(f"{name} checked in this {time_of_day} at {now}")
                with open("Check_in_log.txt", "w") as f:
                    f.write(f"{name} checked in this {time_of_day} at {now}")
                
            if name not in check_out_times  and time_of_day =="afternoon":
                check_out_times[name] = now
                print(f"{name} checked out this {time_of_day} at {now}")
                with open("Check_out_log.txt", "w") as f:
                    f.write(f"{name} checked out this {time_of_day} at {now}")

        # Draw a rectangle around the face
        top, right, bottom, left = face_location
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with the name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Exit the program if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Print the final check-in and check-out times
print("\nFinal check-in times:")
for name, time in check_in_times.items():
    print(f"{name}: {time}")

print("\nFinal check-out times:")
for name, time in check_out_times.items():
    print(f"{name}: {time}")

# Release the webcam and close the window
video_capture.release


Com checked in this morning at 2023-05-06 08:59:25.072248

Final check-in times:
Com: 2023-05-06 08:59:25.072248

Final check-out times:


<function VideoCapture.release>